In [1]:
import os
import bokeh as bk
import numpy as np
import pandas as pd
import seaborn as sns
import math
from bokeh.io import output_file, save, export_svgs, export_png
from bokeh.io import show
from bokeh.layouts import gridplot, row, widgetbox, layout, grid
from bokeh.models import HoverTool, Arrow, NormalHead, ColumnDataSource, LinearColorMapper, FactorRange, DataTable, \
    PanTool, BoxZoomTool, WheelZoomTool, SaveTool, ResetTool, Div, Legend, Panel, Tabs, LabelSet
from bokeh.plotting import figure, show
from bokeh.core.properties import value, field
import panel as pn
import panel.widgets as pnw
pn.extension()



In [2]:
# Serotypes
serotype_names = np.array(["No alignment", "AAV1", "AAV2", "AAV3", "AAV4", "AAV5", "AAV6", 
                "AAV7", "AAV8", "AAV9", "AAV10", "AAV11", "AAV12", "AAV13", 
                "AAVrh8", "AAVrh10", "AAVrh32", "Multiple alignments", "gap"])
serotype_colors = np.array(["gray", "#AA4488", "#4477AA", "#AAAA44", 
                "#AA7744", "#AA4455", "#44AAAA", "#771155", 
                "#114477", "#777711", "#774411", "#771122", 
                "#117777", "#A6CEE3", "#B2DF8A", "#F1B6DA", 
                "#B2ABD2", "black", "white"])

sorter = np.argsort(serotype_names)
idx_del = sorter[np.searchsorted(serotype_names, ["gap", "Multiple alignments", "No alignment"], sorter=sorter)]
serotype_names_del = np.delete(serotype_names, idx_del, 0)
serotype_colors_del = np.delete(serotype_colors, idx_del, 0)

In [3]:
def view_composition(title, serotype_dictionary, serotype_colors, serotype_names, conservation_score=None, include_text = False, plot_width = 1500, max_num_x = 3000):
    
    # reverse the order fot top reps to be in the top
    ### seqs: Serotype list
    ### keys: Library name
    seqs = np.array(list(serotype_dictionary.values())[::-1])
    keys = np.array(list(serotype_dictionary.keys())[::-1])

    N = len(seqs[0])
    S = len(seqs)    
    width = 1.
    
    tmp = [i for s in seqs for i in s]
    colors = [serotype_colors[i] for i in tmp]
    serotypes = [serotype_names[i] for i in tmp]
    
    x = np.arange(1, N + 1)
    y = np.arange(0, S, 1)  
    xx, yy = np.meshgrid(x, y)
    
    gx = xx.flatten()
    gy = yy.flatten() # + 1.5
    recty = gy + 1.5
    h = 1/S
    
    plot_height = len(seqs)*15 + 50
    x_range = bk.models.Range1d(0, N + 1, bounds = 'auto')
    if N > max_num_x:
        viewlen = max_num_x
    else:
        viewlen = N
    view_range = (0, viewlen) 
        
    
    source = bk.models.ColumnDataSource(dict(x=gx, y=gy, recty=recty, colors = colors, serotypes = serotypes))
    
    p = bk.plotting.figure(title = title, 
               plot_width = plot_width, plot_height = plot_height, 
               x_range = view_range, y_range = keys, tools = "xwheel_zoom, xpan, reset", 
               min_border = 0, toolbar_location = 'below')
    
    if include_text == True:
        glyph = bk.models.glyphs.Text(x="x", y="y", text="text", 
                                   text_align='center',
                                   text_color="black",
                                   text_font=value("monospace"))
                                   #text_font_size=fontsize)
    elif include_text == False:
        glyph = bk.models.glyphs.Text(x="x", y="y", text="text", 
                                   text_align='center',
                                   text_color="black",
                                   text_font=value("monospace"))
        
    rects = bk.models.glyphs.Rect(x = "x", y = "recty",  
                 width = 1., height = 1., 
                 fill_color = "colors", 
                 line_color = None, fill_alpha = 1.)
    
    p.add_glyph(source, glyph)
    p.add_glyph(source, rects)
  
    p.grid.visible = False
    p.xaxis.axis_label = 'Nucleotide position'
    p.xaxis.major_label_text_font_style = "bold"
    p.yaxis.minor_tick_line_width = 0
    p.yaxis.major_tick_line_width = 0
    
    # Add a new line in heatmap for conservation scores
    if conservation_score is not None:
        keys = np.append("conservation", keys)
        
        new_x = np.arange(1, N + 1)
        new_y = np.arange(0, S, 1)
        new_xx, new_yy = np.meshgrid(new_x, new_y)

        new_gx = new_xx.flatten()
        new_gy = new_yy.flatten()
        new_recty = new_gy + 0.3 # Adjust the vertical position for the new heatmap

        new_source = bk.models.ColumnDataSource(dict(x=new_gx, y=new_gy, recty=new_recty, data=conservation_score))

        color_mapper = bk.models.LinearColorMapper(palette="YlOrRd9", low=0, high=1) #"Magma256"
        new_rects = bk.models.glyphs.Rect(x="x", y="recty",
                                        width=1., height=1,
                                        fill_color={"field": "data", "transform": color_mapper},
                                        line_color=None, fill_alpha=1.)

        p.add_glyph(new_source, new_rects)

        color_bar = bk.models.ColorBar(color_mapper=color_mapper, location=(0, 0))
        p.add_layout(color_bar, 'right')

    tooltips = [("serotype", "@serotypes"),("nucleotide position", "@x"),]
    p.add_tools(bk.models.HoverTool(tooltips=tooltips))

    # layout = bk.layouts.column(p, height=600, width=1500)
    return p

In [4]:
def view_positional_serotype_abundance(title, positional_serotype_abundance, serotype_names, serotype_colors):
    # def update_data(attrname, old, new):

    #     ### Get the current slider value
    #     smoothing_window_size = smoothing_slider.value
            
    #     smoothed_serotype_abundance = np.copy(positional_serotype_abundance)
    #     for serotype_indx in np.arange(0, len(serotype_names), 1):
    #         abundance_tmp = smoothed_serotype_abundance[serotype_indx]
    #         smoothed_serotype_abundance[serotype_indx] = np.convolve(abundance_tmp, 
    #                                                                 np.ones(smoothing_window_size), 
    #                                                                 'same')/smoothing_window_size
            
    #     ### update the data container
    #     source.data["abundance"] = list(smoothed_serotype_abundance)

    ### The smoothing slider
    smoothing_slider = bk.models.Slider(start = 1, end = 500, value = 100, step = 1, title = "Smoothing length")

    ### Prepare the plotting data
    # placeholder array
    positions_base = np.arange(0, np.shape(positional_serotype_abundance)[1])
    # the main data container
    data_dict = {}
    data_dict["positions"] = [positions_base for _ in np.arange(0, np.shape(positional_serotype_abundance)[0])]
    data_dict["abundance"] = list(positional_serotype_abundance)
    data_dict["color"] = serotype_colors
    data_dict["serotypes"] = serotype_names
    source = bk.plotting.ColumnDataSource(data = data_dict)
    
    # smooth the initially displayed data with default value
    smoothed_serotype_abundance = np.copy(positional_serotype_abundance)
    for serotype_indx in np.arange(0, len(serotype_names), 1):
        abundance_tmp = smoothed_serotype_abundance[serotype_indx]
        smoothed_serotype_abundance[serotype_indx] = np.convolve(abundance_tmp, 
                                                                np.ones(100), 
                                                                'same')/100  
    source.data["abundance"] = list(smoothed_serotype_abundance)

    ### Start plotting
    p = bk.plotting.figure(title = title, 
                            width = 750, height=400,
                            x_axis_label = "Nucleotide position", 
                            y_axis_label = "Serotype abundance")

    p.multi_line(xs = 'positions', ys = 'abundance', source = source,
                                                    line_width = 3, 
                                                    color = 'color',
                                                    legend = 'serotypes')



    p.legend.location = "center"
    #p.legend.click_policy="hide" not working
    leg = p.legend[0]
    p.add_layout(leg,'right') 

    ### Add a tooltip
    tooltips = [("serotype", "@serotypes"),]
    p.add_tools(bk.models.HoverTool(tooltips = tooltips))


    update_data = bk.models.CustomJS(args = dict(source=source, external_data = data_dict), code="""
        var data = source.data;
        var smoothing_window_size = cb_obj.value;
        
        const x_external = external_data["positions"];
        const y_external = external_data["abundance"];
        
        var chunk; var chunk_start; var chunk_end;
        
        data["positions"] = x_external;
        
        console.log("entering loops")
        
        for (var i = 0; i < x_external.length; i++) {

            for (var j = 0; j <= y_external[i].length - Math.floor(smoothing_window_size/2); j++){
            
                if (j - Math.floor(smoothing_window_size/2) < 0)
                    {chunk_start = 0;}
                else
                    {chunk_start = j - Math.floor(smoothing_window_size/2);}
                if (j + Math.floor(smoothing_window_size/2) > y_external[i].length)
                    {chunk_end = y_external[i].length;}
                else
                    {chunk_end = j + Math.floor(smoothing_window_size/2);}
                    
                chunk = y_external[i].slice(chunk_start, chunk_end + 1);
                console.log(chunk)
                data["abundance"][i][j] = chunk.reduce((total, current) => total + current)/chunk.length;
                
            }
        }
        
        
        
        source.change.emit();
        
    """)


    #smoothing_slider.on_change('value', update_data)
    smoothing_slider.js_on_change('value', update_data)


    # show the results
    layout = bk.layouts.column(smoothing_slider, p, width = 750)

    return layout


In [5]:
def get_positional_serotype_abundance_matrix(csv_path):
    chimeric_library_df = pd.read_csv(csv_path, header = None)
    ### Create a dictionary with chimeric_name:[serotype ids]
    variant_description_dictionary = {}
    for chimeric_library_item in chimeric_library_df[0]:
        
        chimeric_library_item_name = chimeric_library_item.split()[0]
        chimeric_library_item_parents = np.array(chimeric_library_item.split()[1:]).astype("int")
        
        variant_description_dictionary[chimeric_library_item_name] = chimeric_library_item_parents

    ### 2-D array of chimeric composition
    chimeric_composition_array = np.array(list(variant_description_dictionary.values()))

    ### Initialize the abundance matrix ---> rows:serotypes, columns:positional abundance 
    positional_serotype_abundance = np.zeros((len(serotype_names), np.shape(chimeric_composition_array)[1]))

    ### Populate the abundance matrix
    for serotype_indx in np.arange(0, len(serotype_names), 1):
        positional_serotype_abundance[serotype_indx] = np.sum(chimeric_composition_array == serotype_indx, axis = 0)
    
    # Remove gap, no alignment, multiple alignment
    positional_serotype_abundance_del = np.delete(positional_serotype_abundance, idx_del, 0)

    return positional_serotype_abundance_del

In [6]:
def get_vd_dictionary(csv_path):
    chimeric_library_df = pd.read_csv(csv_path, header = None)
    ### Create a dictionary with chimeric_name:[serotype ids]
    variant_description_dictionary = {}
    for chimeric_library_item in chimeric_library_df[0]:
        
        chimeric_library_item_name = chimeric_library_item.split()[0]
        chimeric_library_item_parents = np.array(chimeric_library_item.split()[1:]).astype("int")
        
        variant_description_dictionary[chimeric_library_item_name] = chimeric_library_item_parents
    return variant_description_dictionary

Predicted

In [458]:
# csv_path = "../../../../../hafoe/data_v4/benchmarking/hafoe_out_1_100_10_sum/files/Chimeric_rep_predicted_labels.csv"

# pred_df_initial = pd.read_csv(csv_path)
# pred_df = pred_df_initial.iloc[:,0:2]
# pred_df = pd.DataFrame(pred_df.iloc[:,0] +" "+ pred_df.iloc[:,1])
# ### Create a dictionary with chimeric_name:[serotype ids]
# variant_description_dictionary_pred = {}
# for chimeric_library_item in pred_df[0]:
    
#     chimeric_library_item_name = chimeric_library_item.split()[0]
#     chimeric_library_item_parents = np.array(chimeric_library_item.split()[1:]).astype("int")
    
#     variant_description_dictionary_pred[chimeric_library_item_name] = chimeric_library_item_parents

# #add 18s (gap) at the end to make variant lists of same length
# max_len = max([len(x) for x in variant_description_dictionary_pred.values()])
# for k,v in zip(variant_description_dictionary_pred.keys(), variant_description_dictionary_pred.values()):
#     variant_description_dictionary_pred[k] = np.concatenate((v,np.repeat([18], max_len-len(v))))

In [7]:
pred_csv_path = "../../../../../hafoe/data_v4/benchmarking/hafoe_out_1_100_10_sum/files/variant_description/chimeric_lib_representatives/chimeric_lib_representatives_variant_description_top20_msa.csv"
variant_description_dictionary_pred = get_vd_dictionary(pred_csv_path)
positional_serotype_abundance_pred = get_positional_serotype_abundance_matrix(pred_csv_path)

conservation_score_df_pred = pd.read_csv("../../../../../hafoe/data_v4/benchmarking/hafoe_out_1_100_10_sum/files/top20_rep_msa_conservation_score.csv")
conservation_score_pred = np.array(conservation_score_df_pred.x)

In [460]:
p1_pred = view_composition("Predicted compositions of synthetic chimeric library representatives", 
                 variant_description_dictionary_pred, 
                 serotype_colors, 
                 serotype_names,
                 plot_width=740)
p2_pred = view_positional_serotype_abundance("Predicted positional serotype abundance of synthetic chimeric library representatives",
                                            positional_serotype_abundance_pred, 
                                            serotype_names_del, 
                                            serotype_colors_del)
# pn.pane.Bokeh(p2_pred)

In [8]:
p = view_composition("Predicted compositions of synthetic chimeric library representatives",
                     variant_description_dictionary_pred, 
                     conservation_score=conservation_score_pred,
                     serotype_colors=serotype_colors,
                     serotype_names=serotype_names,
                     plot_width=740)
pn.pane.Bokeh(p)

Bokeh(Figure)

True

In [462]:
# csv_path = "../../../../../hafoe/data_v4/input_files/data1/Chimeric_lib_simulated_labels.csv"
# query_seq = list(variant_description_dictionary_pred.keys())
# query_seq = [x.split("_", 1)[0] for x in query_seq]

# true_df = pd.read_csv(csv_path).iloc[:,0:2]
# true_df = true_df[true_df['X'].isin(query_seq)]
# true_df = pd.DataFrame(true_df.iloc[:,0] +" "+ true_df.iloc[:,1])

# ### Create a dictionary with chimeric_name:[serotype ids]
# variant_description_dictionary_true = {}
# for chimeric_library_item in true_df[0]:
    
#     chimeric_library_item_name = chimeric_library_item.split()[0]
#     chimeric_library_item_parents = np.array(chimeric_library_item.split()[1:]).astype("int")
    
#     variant_description_dictionary_true[chimeric_library_item_name] = chimeric_library_item_parents

# max_len = max([len(x) for x in variant_description_dictionary_pred.values()])
# #add 18s (gap) at the end to make variant lists of same length
# for k,v in zip(variant_description_dictionary_true.keys(), variant_description_dictionary_true.values()):
#     s = pred_df_initial[pred_df_initial.X.str.match(k+'.*')]['Start_orf']-1
#     e = pred_df_initial[pred_df_initial.X.str.match(k+'.*')]['End_orf']
#     variant_description_dictionary_true[k] = v[int(s):int(e)]
#     variant_description_dictionary_true[k] = np.concatenate((variant_description_dictionary_true[k],np.repeat([18], max_len-len(variant_description_dictionary_true[k]))))

In [9]:
true_csv_path = "../../../../../hafoe/data_v4/input_files/data1/Chimeric_lib_simulated_labels_top20_msa.csv"
variant_description_dictionary_true = get_vd_dictionary(true_csv_path)
positional_serotype_abundance_true = get_positional_serotype_abundance_matrix(true_csv_path)

conservation_score_df_true = pd.read_csv("../../../../../hafoe/data_v4/input_files/data1/Chimeric_lib_simulated_initial_orf_top20_conservation_score.csv")
conservation_score_true = np.array(conservation_score_df_true.x)

In [464]:
p1_true = view_composition("True compositions of synthetic chimeric library representatives", 
                 variant_description_dictionary_true, 
                 serotype_colors, 
                 serotype_names,
                 plot_width=740)
p2_true = view_positional_serotype_abundance("True positional serotype abundance of synthetic chimeric library representatives",
                                            positional_serotype_abundance_true, 
                                            serotype_names_del, 
                                            serotype_colors_del)
# pn.pane.Bokeh(p2_true)

In [11]:
p = view_composition("True compositions of synthetic chimeric library representatives",
                    variant_description_dictionary_true, 
                    conservation_score=conservation_score_true,
                    serotype_colors=serotype_colors,
                    serotype_names=serotype_names,
                    plot_width=740)
pn.pane.Bokeh(p)

Bokeh(Figure)

Save in a html report

In [465]:
def bokeh_composite(title, figure_layout, filename):

    output_file(title + ".html", title=title)

    p = layout(figure_layout, sizing_mode="scale_width")

    save([p], filename=filename)

In [466]:
bokeh_composite("variant_composition_accuracy",  
                figure_layout = [bk.layouts.column(bk.layouts.row(p2_true, p2_pred, width=1500), bk.layouts.row(p1_true, p1_pred, width=1500))], 
                filename = os.path.join("variant_composition_accuracy.html"))